## Using Python to process data that's too large to fit into memory

I generated some sample data into a database using [Mockaroo](https://www.mockaroo.com/) that includes bg values for different users throughout the day. 

Although the data is only 600 rows, I'll use this opportunity to process it chunkwise in python as if it were data that could not fit in memory. Let's get a quick look at the data first.

### Database file with pandas

In [256]:
import pandas as pd
import sqlite3

#Connect to our database
con= sqlite3.connect('bgTable.db')

#Read the first few lines from our table
table0= pd.read_sql_query('SELECT * FROM bgTable LIMIT 3', con)
table0.head()

,id,bgVal,time
0,1,230,15:52
1,2,80,19:22
2,3,347,8:13


Now that we can see our table, we'll need to define the query that we'd like to apply to the table. We want to add a new column to our data that labels our row based on the bgValues value. It can fall into three groups: 1) Less than or equal to 80 2) between 81 and 250 and 3) Greater than 250. 

In [257]:
#binItems query has nested 'case when' statements to create a new
#column called 'bin' that includes a label for each row depending
#on the value of the bgVal column
binItems= '''
SELECT *, 
CASE WHEN bgVal <=80 THEN 'below'  
     ELSE CASE WHEN bgVal <=250 THEN 'in_range'  
          ELSE CASE WHEN bgVal >250 THEN 'above' 
          END 
     END
END AS 'bin'
FROM bgTable
'''

Now that we have the query, we'll apply it to bgTable. This will add an extra column ('bin') for the bin each row belongs to ('below','in_range', and 'above'). Again, we'll perform this operation in chunks (since we're assuming that the table is too large to load into memory). 

The code in the next cell does the following: 1) loads a chunk of our table from our database 3 rows at a time, 2) performs the 'binItems' query on each chunk and 3) appends the resulting chunks to a new table called 'bgTableBinned' that has a new 'bin' column. 

In [258]:
#Changing this query to sqlite directly
#pd.read_sql_query('create table bgTableBinned2(id INTEGER, bgVal VARCHAR, time VARCHAR, bin VARCHAR)', con)

c= con.cursor()
c.execute('create table bgTableBinned2(id INTEGER, bgVal VARCHAR, time VARCHAR, bin VARCHAR)')

In [261]:
#We'll first create a new table in our database that
#we'll append new items to
#pd.to_sql('bgTableBinned3',con, index= False, if_exists= 'append')
#pd.read_sql_query('create table bgTableBinned2(id INTEGER, bgVal VARCHAR, time VARCHAR, bin VARCHAR)', con)
#CREATE TABLE t(x INTEGER PRIMARY KEY ASC, y, z);

for query_chunk in pd.read_sql_query(binItems, con, chunksize=3):
    print query_chunk 
    query_chunk.to_sql('bgTableBinned2',con, if_exists= 'append', index= False)

#Interesting, using pd.read_sql_query allows me to create a table but I keep getting a
#"NoneType object is not iterable" error directed at that line. Should i just be using
#query_chunk.to_sql to wire with parameters changed?
#If I run the .tosql command I get a 'SQL logic error or missing database' error
#However, if I look at the db on dbBrowser, the table HAS been created and the 
#first three rows have been written. If I run this same cell again it works fine
#but you can tell something funky was up cause there are 607 rows instead of 7
#(why not 603?)
#looks like it wrote rows 123, then appended 1234, then appended 1234...600 weird

#Showed this to the data engineers and this 'hiccup' of 1234, 1234...600 didn't make sense
#it didn't crash and reset itself. Could this be a bug?
#Also, not a solution but it was suggested that instead of writing a whole table I could 
#update original table with a single column instead?
#Also, since I have the con object, maybe make the table using sqlite3 instead of pandas
#and just do the writing part in pandas?

   id  bgVal   time       bin
0   1    230  15:52  in_range
1   2     80  19:22     below
2   3    347   8:13     above
   id  bgVal   time       bin
0   4    220   5:53  in_range
1   1    230  15:52  in_range
2   2     80  19:22     below
   id  bgVal  time       bin
0   3    347  8:13     above
1   4    220  5:53  in_range
2   5    305  0:36     above
   id  bgVal   time       bin
0   6    148  18:16  in_range
1   7    269  19:19     above
2   8    193  16:24  in_range
   id  bgVal   time       bin
0   9    305   3:12     above
1  10    120  12:28  in_range
2  11    188  13:28  in_range
   id  bgVal   time       bin
0  12    135  20:49  in_range
1  13    134   0:04  in_range
2  14    122  17:05  in_range
   id  bgVal  time       bin
0  15    297  1:17     above
1  16     58  2:47     below
2  17    202  4:29  in_range
   id  bgVal   time       bin
0  18     75   3:33     below
1  19    137   7:37  in_range
2  20    131  21:46  in_range
   id  bgVal   time       bin
0  21    170  21:5

In [5]:
#Save and close connection to database
con.close()

In [6]:
#Reopen connection and read new table

#Connect to our database with new table
conNew= sqlite3.connect('bgTable.db')

#Read the first few lines from our new table
newTable= pd.read_sql_query('SELECT * FROM bgTableBinned LIMIT 3', conNew)
newTable.head()

,id,bgVal,time,bin
0,1,230,15:52,in_range
1,2,80,19:22,below
2,3,347,8:13,above


In [7]:
conNew.close()

Note: For some reason, when I run cell #4 the first time, I get a logic error that says database doesn't exist. However, when I run the same cell again, it works, as do the remaining cells. I'll need to explore this a little further tomorrow (maybe I need to create the new table out of the loop first?), but it looks like for the first run, my workflow is doing what I need it to do. 

### Buffering with a CSV file and Numpy

I created a csv file of the data in bgTable in order to perform the same operations with this file format. I could so something similar to what I did with the db file using pandas, but I'll start by avoiding external modules.

The key argument that we'll use is the 'buffering' argument for the 'open' function. Before that though, let's define the function that will bin the values into our three bins once more and test the operations on a separate table before we insert it into the chunk loop.

In [9]:
#Let's try operating on the sample table from the first line in our notebook
table0

,id,bgVal,time
0,1,230,15:52
1,2,80,19:22
2,3,347,8:13


In [136]:
#We'll need to import numpy and turn this dataframe into an array
#Let's verify that we have a three column array
import numpy as np

arrayTable= np.array(table0)
arrayTable 

array([[1, 230, u'15:52'],
       [2, 80, u'19:22'],
       [3, 347, u'8:13']], dtype=object)

In [142]:
def binFxn(myArray):
    '''This function will be passed and applied to every row in 
    my chunk (to avoid iterating through a for loop) and will
    create create a new column that labels a row based on the
    value in its second column as below, in_range, or above'''
    if  myArray[1] <= 80: 
        return 'below'
    elif myArray[1] <= 250 and myArray[1] > 80:
        return 'in_range'
    elif myArray[1] >= 250:
        return 'above'

In [143]:
#We'll apply the binFxn to each row in our table and 
#the output will be a new column that we'll call newArray
newArray= np.apply_along_axis(binFxn, 1, arrayTable)
newArray= newArray.reshape(3,1)

In [144]:
#We'll append this new array to our origina array
#that should give us a 4 column table
np.append(arrayTable, newArray, axis=1)

array([[1, 230, u'15:52', 'in_range'],
       [2, 80, u'19:22', 'below'],
       [3, 347, u'8:13', 'above']], dtype=object)

Cool! Seems to be doing what we want it to do. Now we just need to perform this action on every chunk, and then write/append that to a new file. Let's try implementing that on a version of bgTable that comes in csv format.

In [147]:
#Initialize new file to append the new table data into
newTable= open('bgTableNew.csv','a')

#We'll use the buffering argument to limit the chunk size
#buffering=1 mb is the smallest unit we can choose
with open('bgTable.csv', buffering=1) as myFile:
    for chunk in myFile:
        arrayTable= np.array(chunk)
        newArray= np.apply_along_axis(binFxn, 1, arrayTable)
        newArray= newArray.reshape(3,1)
        
        newTable.write(np.append(arrayTable, newArray, axis=1))

ValueError: axis must be less than arr.ndim; axis=1, rank=0.

Will work on this and previous day's method more tomorrow.